# Introduction

This notebook demonstrates the use of a language model, specifically the "meta-llama/Llama-3.2-1B-Instruct", for generating text and embeddings. It begins by setting up the model and tokenizer using the Hugging Face Transformers library, ensuring that the pad token is correctly defined. The notebook then illustrates how to generate text using the model in both streaming and non-streaming modes. It applies a chat template to user messages, moves inputs to a GPU if available, and generates outputs with a specified maximum number of tokens. The generated text is cleaned to remove system messages, and the time taken for generation is displayed.
In addition to text generation, the notebook explores embeddings using the SentenceTransformer library. It encodes words and sentences to compute cosine similarity matrices, which are visualized to show relationships between different words and sentences. The notebook also demonstrates the concept of Retrieval Augmented Generation (RAG) by encoding a user's question and sorting sentences based on their similarity to the question. This approach helps in retrieving relevant information from a text corpus. Finally, the notebook sets up a pipeline for generating responses to user queries, showcasing the integration of text generation and retrieval techniques.

## Environment Variables
we will need to use Environment Variables:
- HF_TOKEN is you huggingface token, you may generate one on this url: https://huggingface.co/settings/tokens

We will use Llama-3.2-1B-Instruct. In order to be able to download it you will need to accept the terms of use on the following link:

https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct

You can check if you have been granted on:

https://huggingface.co/settings/gated-repos

# Import libraries

https://huggingface.co/docs/transformers/index

In [43]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, pipeline
import os
import torch
import time
import re

In [44]:
# If HF_TOKEN is not set, prompt the user to enter it securely
# import getpass
# os.environ['HF_TOKEN'] = getpass.getpass("Enter the value for HF_TOKEN: ")

## HF_HOME
is the directory where you want to save models' weights. export $HF_HOME as well.

In [45]:
# E.g. on colab do:
# os.environ["HF_HOME"] = "/content/my_huggingface_cache"

# The Transformers Library

## Download Models

In [46]:
my_model = "meta-llama/Llama-3.2-1B-Instruct"

In [47]:
tokenizer = AutoTokenizer.from_pretrained(my_model,
                                          token=os.environ["HF_TOKEN"],
                                          cache_dir=os.environ["HF_HOME"])

In [48]:
model = AutoModelForCausalLM.from_pretrained(   my_model,
                                                token=os.environ["HF_TOKEN"],
                                                cache_dir=os.environ["HF_HOME"],
                                                device_map="auto")

## pad token

In [ ]:
# Depending on the model, the pad token might not be defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Pad token was None, so it was set to eos token.")

## Streamer for model.generate and pipeline

In [50]:
streamer = TextStreamer(tokenizer)

## Messages

In [51]:
system_instructions = f"You are a helpful assistant."
my_messages = [{"role": "system", "content": system_instructions}]
my_prompt = """What is an LLM?"""
my_messages.append({"role": "user", "content": my_prompt})

## Streaming Model

### Apply chat template to messages and return tensors

In [ ]:
inputs = tokenizer.apply_chat_template(my_messages, return_tensors="pt")
print(type(inputs)) # <class 'torch.Tensor'>
attention_mask = (inputs != tokenizer.pad_token_id).long()

### Move inputs to GPU if available

In [53]:
if torch.cuda.device_count()>0:
    inputs = inputs.to("cuda")
    attention_mask = attention_mask.to("cuda")
    print("Inputs and Attention Mask transfered to CUDA")

In [ ]:
t1 = time.time()
MAXIMUM_TOKENS = 128
outputs = model.generate(inputs,
                         streamer=streamer,
                         pad_token_id=tokenizer.eos_token_id,
                         attention_mask=attention_mask,
                         max_new_tokens=MAXIMUM_TOKENS)
t2 = time.time()

In [ ]:
print(type(outputs)) # <class 'torch.Tensor'>

### Clean the sesponse

In [ ]:
# To ommit <|begin_of_text|><|start_header_id|>system<|end_header_id|> we use:
generated_text = tokenizer.decode(outputs[0],
                                  skip_special_tokens=True,
                                  clean_up_tokenization_spaces=True)
print(f"{generated_text}\n\n{(t2-t1)/60:.2f} minutes")

In [ ]:
print(type(generated_text)) # <class 'str'>

In [ ]:
# To omit the system message we use:
cleaned_text = re.sub(r"^.*?assistant\n\n", "", generated_text, flags=re.DOTALL)
print(cleaned_text + "\n\n" + f"{(t2-t1)/60:.2f} minutes")

## Inference (without steaming)

In [ ]:
t1 = time.time()
MAXIMUM_TOKENS = 128
outputs = model.generate(inputs,
                         pad_token_id=tokenizer.eos_token_id,
                         attention_mask=attention_mask,
                         max_new_tokens=MAXIMUM_TOKENS)
t2 = time.time()
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text + "\n\n" + f"{(t2-t1)/60:.2f} minutes")

## The Pipeline object

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                device_map="auto")
t1 = time.time()
MAXIMUM_TOKENS = 128
outputs = pipe(my_messages,
               max_new_tokens=MAXIMUM_TOKENS,
               pad_token_id=pipe.tokenizer.eos_token_id,
               streamer=streamer)
t2 = time.time()

In [ ]:
# In pipeline outputs (not in model.generate) we have the "generated_text" attribute:
print(outputs[0]["generated_text"][-1]['content'] + "\n\n" + f"{(t2-t1)/60:.2f} minutes")
# [{'generated_text': [{'role': 'system', 'content': 'You are a helful assistant.'},
#                       {'role': 'user', 'content': "...
type(outputs) # <class 'list'>
type(outputs[0]) # <class 'dict'>

# Embeddings

## Import libraries

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
import os
if torch.cuda.device_count()>0:
    my_device = "cuda"
    print(f"You have {torch.cuda.device_count()} GPUs available.")
else:
    my_device = "cpu"
    print("You have no GPUs available. Running on CPU.")

## The SentenceTransformer object

In [61]:
embeddings_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2',
                                       token=os.environ["HF_TOKEN"],
                                       cache_folder=os.environ["HF_HOME"],
                                       device=my_device)

## Function to visualizing the similarity matrix

In [62]:
import matplotlib.pyplot as plt
def visualize_similarity_matrix(similarity_matrix, items_labels, mat_size=5):
    for i in range(similarity_matrix.shape[0]):
        similarity_matrix[i,i] = 0
    plt.figure(figsize=(mat_size, mat_size))
    plt.imshow(similarity_matrix, interpolation='nearest', cmap='viridis')
    plt.colorbar(label="Cosine Similarity")
    plt.xticks(ticks=np.arange(len(items_labels)), labels=items_labels, rotation=90, fontsize=8)
    plt.yticks(ticks=np.arange(len(items_labels)), labels=items_labels, fontsize=8)
    plt.title("Cosine Similarity Matrix", fontsize=12)
    plt.tight_layout()
    plt.show()

## Test Embeddings - unrelated words

In [ ]:
word_list = ["reciprocal", "obfuscate", "hyperbolic", "tensor"]
word_embeddings = embeddings_model.encode(word_list)
word_embeddings

In [ ]:
cosine_similarities = cosine_similarity(word_embeddings)
print("Cosine Similarity Matrix:")
print(cosine_similarities)
visualize_similarity_matrix(cosine_similarities, word_list)

## Test Embeddings - related words

In [ ]:
word_list = ["book", "book!", "publication", "article"]
word_embeddings = embeddings_model.encode(word_list)
cosine_similarities = cosine_similarity(word_embeddings)
print("Cosine Similarity Matrix:")
print(cosine_similarities)
visualize_similarity_matrix(cosine_similarities, word_list)

## Calculate normalized mean values of embeddings

In [ ]:
mean_embeddings = np.mean(np.abs(word_embeddings), axis=1)
print("Normalized Mean values of embeddings:", mean_embeddings)
std_embeddings = np.std(word_embeddings, axis=1)
print("Standard Deviation of embeddings:", std_embeddings)
norm_embeddings = np.linalg.norm(word_embeddings, axis=1)
print("Norm of embeddings:", norm_embeddings)

## Generate random vectors with the same mean and std

In [ ]:
random_vectors = np.random.normal(loc=np.mean(word_embeddings),
                                  scale=np.std(word_embeddings),
                                  size=word_embeddings.shape)
mean_random_vectors = np.mean(np.abs(random_vectors), axis=1)
print("Normalized Mean values of random vectors:", mean_random_vectors)
std_random_vectors = np.std(random_vectors, axis=1)
print("Standard Deviation of random vectors:", std_random_vectors)
norm_random_vectors = np.linalg.norm(random_vectors, axis=1)
print("Norm of random vectors:", norm_random_vectors)

In [ ]:
print("Cosine Similarity Matrix random vectors:")
cosine_similarities = cosine_similarity(random_vectors)
print(cosine_similarities)
visualize_similarity_matrix(cosine_similarities, ["Random Vector 1", "Random Vector 2", "Random Vector 3"])

## car ~ vehicle + motorcycle - bike

In [ ]:
sentences = ["car", "vehicle", "motorcycle", "bike"]
embeddings = embeddings_model.encode(sentences)
print(cosine_similarity(embeddings[0].reshape(1, -1), (embeddings[1] + embeddings[2] - embeddings[3]).reshape(1, -1))[0, 0])

## Greece ~ Athens + Italy - Rome

In [ ]:
sentences = ["Greece", "Athens", "Italy", "Rome"]
embeddings = embeddings_model.encode(sentences)
print(cosine_similarity((embeddings[0]).reshape(1, -1), (embeddings[1]+embeddings[2]-embeddings[3]).reshape(1, -1))[0, 0])

So embeddings work!

## Sentence embeddings

In [ ]:
my_sentences = [
    # Interrelated sentences - group 1
    "The data is preprocessed to remove noise and outliers.",
    "Noise and outliers are eliminated during data preprocessing.",
    "Preprocessing cleans the data by filtering out noise and irregularities.",

    # Interrelated sentences - group 2
    "Paris is the capital of France.",
    "Athens is the capital of Greece.",
    "Rome is the capital of Italy."
]
my_embeddings = embeddings_model.encode(my_sentences)
similarity_matrix = cosine_similarity(my_embeddings)
print(similarity_matrix)
visualize_similarity_matrix(similarity_matrix, my_sentences, mat_size=8)

# Retrieval Augmented Generation (RAG)

In [72]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
if torch.cuda.device_count()>0:
    my_device = "cuda"
    print(f"You have {torch.cuda.device_count()} GPUs available.")
else:
    my_device = "cpu"
    print("You have no GPUs available. Running on CPU.")

## Embeddings Model

In [74]:
embeddings_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', token=os.environ["HF_TOKEN"],
                                       cache_folder=os.environ["HF_HOME"], device=my_device)

## Text for retrieval

In [75]:
my_text = """
This notebook demonstrates the use of a language model, specifically the "meta-llama/Llama-3.2-1B-Instruct", for generating text and embeddings. It begins by setting up the model and tokenizer using the Hugging Face Transformers library, ensuring that the pad token is correctly defined. The notebook then illustrates how to generate text using the model in both streaming and non-streaming modes. It applies a chat template to user messages, moves inputs to a GPU if available, and generates outputs with a specified maximum number of tokens. The generated text is cleaned to remove system messages, and the time taken for generation is displayed.
In addition to text generation, the notebook explores embeddings using the SentenceTransformer library. It encodes words and sentences to compute cosine similarity matrices, which are visualized to show relationships between different words and sentences. The notebook also demonstrates the concept of Retrieval Augmented Generation (RAG) by encoding a user's question and sorting sentences based on their similarity to the question. This approach helps in retrieving relevant information from a text corpus. Finally, the notebook sets up a pipeline for generating responses to user queries, showcasing the integration of text generation and retrieval techniques.
"""

In [76]:
my_sentences = my_text.split('.')

In [ ]:
my_sentences[:3]

In [ ]:
my_embeddings = embeddings_model.encode(my_sentences)
print(my_embeddings.shape)

## Encode user's question

In [79]:
my_question = "What is this notebook about?"
my_question_embedding = embeddings_model.encode([my_question])

## Sort sentences based on the similarity to the question embedding

In [ ]:
similarity_to_question = cosine_similarity(my_question_embedding, my_embeddings).flatten()
sorted_indices = similarity_to_question.argsort()[::-1]  # Sort in descending order
print(sorted_indices)

## Get sorted sentences

In [ ]:
sorted_sentences = [my_sentences[i] for i in sorted_indices]
print("Sorted sentences based on cosine similarity to the question:")
for i, sentence in enumerate(sorted_sentences):
    print("-"*100)
    print(f"Sentence {i+1}, similarity: {similarity_to_question[sorted_indices[i]]:.2f}")
    print(sentence)

## Setup messages

In [ ]:
nof_keep_sentences = 3
system_instructions = f"You are a helful assistant."
my_messages = [{"role": "system", "content": system_instructions}]
my_prompt = f"Use the following sentences:"
for sentence in sorted_sentences[:nof_keep_sentences]:
    my_prompt += f"\n{sentence}"
my_prompt += f"\n\nAnswer the question:\n\n'{my_question}'"
my_prompt += "\n\nStart your reply with the word Hello!"
my_messages.append({"role": "user", "content": my_prompt})
my_prompt

## Answer the question

In [83]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer
import os
my_model = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(my_model,
                                          token=os.environ["HF_TOKEN"],
                                          cache_dir=os.environ["HF_HOME"])
model = AutoModelForCausalLM.from_pretrained(   my_model,
                                                token=os.environ["HF_TOKEN"],
                                                cache_dir=os.environ["HF_HOME"],
                                                device_map="auto")

In [ ]:
# Depending on the model, the pad token might not be defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Pad token was None, so it was set to eos token.")

streamer = TextStreamer(tokenizer)

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                device_map="auto")
MAXIMUM_TOKENS = 128
outputs = pipe(my_messages,
               max_new_tokens=MAXIMUM_TOKENS,
               pad_token_id=pipe.tokenizer.eos_token_id,
               streamer=streamer)

In [ ]:
print(outputs[0]["generated_text"][-1]['content'])